In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.datasets import reuters
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier, KerasRegressor
from math import floor
from sklearn.metrics import make_scorer, accuracy_score, log_loss
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

# Set random seed
np.random.seed(20712124)

data_train = pd.read_csv("../preprocessing/preprocesed_variables_train.csv")
important_variables = pd.read_csv("../preprocessing/important.csv")

X = data_train.drop(columns=['health'])
X = X[important_variables['0']]
y = data_train['health']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=20712124, stratify = y)

# Create correlation matrix
corr_matrix = X_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

X_train = X_train.drop(X_train[to_drop], axis=1)
X_test = X_test.drop(X_test[to_drop], axis=1)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
temp_m = SelectFromModel(lsvc, prefit=True)
X_new = temp_m.transform(X_train)
X_new_test = temp_m.transform(X_test)
cols = temp_m.get_support(indices=True)
X_new = pd.DataFrame(X_new, columns = cols)
X_new_test = pd.DataFrame(X_new_test, columns = cols)


E:\Anaconda\envs\tf_gpu\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
E:\Anaconda\envs\tf_gpu\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [10]:
X_new

array([[0.        , 0.        , 0.5       , ..., 0.46666667, 0.83333333,
        1.        ],
       [0.25      , 0.25      , 0.5       , ..., 0.4       , 0.83333333,
        0.77777778],
       [0.5       , 0.25      , 0.75      , ..., 0.86666667, 1.        ,
        1.        ],
       ...,
       [0.5       , 0.5       , 0.25      , ..., 0.93333333, 0.33333333,
        0.22222222],
       [0.25      , 0.25      , 0.5       , ..., 0.26666667, 1.        ,
        1.        ],
       [1.        , 1.        , 0.5       , ..., 0.46666667, 0.33333333,
        0.88888889]])

In [12]:
def neg(fn):
    return lambda x: -fn(x)

score_acc = make_scorer(accuracy_score)
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs,
             layers1, layers2, dropout, dropout_rate ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', 'relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def nn_cl_fun():
        opt = optimizerD[optimizer]

        inputs = tf.keras.Input(shape=(len(X_new.columns,))) # input with 934 
        x = inputs
        for i in range(layers1):
            x = Dense(neurons, activation=activation)(x)
        if dropout < 0.5:
            x = Dropout(dropout_rate, seed=123)(x)
        for i in range(layers2):
            x = Dense(neurons, activation=activation)(x)
        outputs = Dense(5, activation='softmax')(x) #output
        nn = tf.keras.Model(inputs=inputs, outputs=outputs, name="base_model")

        nn.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(model=nn_cl_fun, epochs=epochs, batch_size=batch_size,
                         verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_new, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [13]:
import warnings
def action_with_warnings():
    warnings.warn("should not appear")

with warnings.catch_warnings(record=True):
    action_with_warnings()

# Set paramaters
params_nn ={
    'neurons': (25, 60),
    'activation':(0, 0.4),
    'optimizer':(3.5,3.9),
    'learning_rate':(0.01, 0.3),
    'batch_size':(400, 500),
    'epochs':(4, 72),
    'layers1':(2,3),
    'layers2':(2,3),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=100)
nn_bo.maximize(init_points=30, n_iter=5)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.4025   |  0.2174   |  427.8    |  0.4245   |  0.2534   |  4.321    |  2.122    |  2.671    |  0.2495   |  29.78    |  3.73     |
|  2        |  0.4509   |  0.3565   |  420.9    |  0.1853   |  0.03251  |  18.94    |  2.979    |  2.812    |  0.05986  |  53.57    |  3.61     |
|  3        |  0.4482   |  0.1727   |  494.0    |  0.8176   |  0.1008   |  15.93    |  2.373    |  2.006    |  0.0832   |  52.85    |  3.506    |
|  4        |  0.4292   |  0.2395   |  460.4    |  0.1051   |  0.1146   |  6.48     |  2.89     |  2.981    |  0.02738  |  56.17    |  3.731    |
|  5        |  0.4146   |  0.297    |  463.0    |  0.5818   |  0.006132 |  18.28    |  2.545    |  2.769    |  0.0827   |  3

In [35]:
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential','relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]
params_nn_

{'activation': 'relu',
 'batch_size': 419,
 'dropout': 0.5218717978245897,
 'dropout_rate': 0.14874659577110114,
 'epochs': 71,
 'layers1': 3,
 'layers2': 2,
 'learning_rate': 0.13680796401982245,
 'neurons': 40,
 'optimizer': <keras.optimizer_v2.adagrad.Adagrad at 0x1cb35e14ac0>}

In [36]:
def nn(**kwargs):
    opt = kwargs['optimizer']

    inputs = tf.keras.Input(shape=(len(X.columns,))) # input with 934 
    x = inputs
    for i in range(kwargs['layers1']):
        x = Dense(kwargs['neurons'], activation=kwargs['activation'])(x)
    if kwargs['dropout'] < 0.5:
        x = Dropout(kwargs['dropout_rate'], seed=123)(x)
    for i in range(kwargs['layers2']):
        x = Dense(kwargs['neurons'], activation=kwargs['activation'])(x)
    outputs = Dense(5, activation='softmax')(x) #output
    nn = tf.keras.Model(inputs=inputs, outputs=outputs, name="base_model")
    nn.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 
    nn.fit(X_train.to_numpy(), y_train.to_numpy(), epochs = kwargs['epochs'],batch_size=kwargs['batch_size'])
    return nn

model = nn(**params_nn_)
model.evaluate(X_test, y_test, verbose =2 )

Epoch 1/71
25/25 [==============================] - 0s 5ms/step - loss: 1.4174 - accuracy: 0.3821
Epoch 2/71
25/25 [==============================] - 0s 4ms/step - loss: 1.3792 - accuracy: 0.3907
Epoch 3/71
25/25 [==============================] - 0s 4ms/step - loss: 1.3574 - accuracy: 0.4008
Epoch 4/71
25/25 [==============================] - 0s 4ms/step - loss: 1.3302 - accuracy: 0.4191
Epoch 5/71
25/25 [==============================] - 0s 5ms/step - loss: 1.3152 - accuracy: 0.4279
Epoch 6/71
25/25 [==============================] - 0s 4ms/step - loss: 1.2969 - accuracy: 0.4322
Epoch 7/71
25/25 [==============================] - 0s 5ms/step - loss: 1.2902 - accuracy: 0.4358
Epoch 8/71
25/25 [==============================] - 0s 4ms/step - loss: 1.2883 - accuracy: 0.4335
Epoch 9/71
25/25 [==============================] - 0s 4ms/step - loss: 1.2710 - accuracy: 0.4466
Epoch 10/71
25/25 [==============================] - 0s 4ms/step - loss: 1.2661 - accuracy: 0.4433
Epoch 11/71
25/25 [

[1.2246575355529785, 0.4544540345668793]

In [38]:
data_test = pd.read_csv("../preprocessing/preprocesed_variables_test.csv")
og_test = pd.read_csv("../../data/test.csv")
unique_ids = og_test['uniqueid']

data_test = data_test.drop(columns='health')
data_test = data_test[important_variables['0']]

test_pred_proba = model.predict_proba(data_test)
test_pred = pd.DataFrame({'p':model.predict(data_test)})

test_pred.to_csv("temp.csv", index = False)
preds = pd.DataFrame(test_pred_proba, columns=['p1','p2','p3','p4','p5'])
preds.insert(0, 'uniqueid', unique_ids)

preds.to_csv("xgb_pred.csv", index = False)